In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dropout, LayerNormalization, Dense, TimeDistributed, RepeatVector, MultiHeadAttention, Concatenate
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from tensorflow.keras import mixed_precision
import keras_tuner as kt
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import os

# 2. Configuração de Recursos Computacionais e Precisão Mista

a. Configuração do TensorFlow para Utilizar a GPU

b. Ativar Mixed Precision

In [ ]:
# Verifique se a GPU está disponível
print("GPUs disponíveis:", tf.config.list_physical_devices('GPU'))

# Configurar o crescimento da memória da GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Configuração de memória da GPU concluída.")
    except RuntimeError as e:
        print(e)
else:
    print("GPU não detectada. O treinamento será realizado na CPU.")

# 3. Pré-processamento dos Dados

Utilizaremos o MinMaxScaler para escalar os dados entre 0 e 1, o que é adequado para redes LSTM devido às funções de ativação utilizadas.

Se y_train e y_test contêm apenas a variável de interesse (ws100), o escalonamento direto funciona bem. Caso contrário, ajuste o escalonamento conforme necessário para preservar as outras features.

In [ ]:
# Definição do Scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Escalonamento dos dados de treino
num_train_samples = X_train.shape[0] * X_train.shape[1]
X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])  # [num_samples * sequence_length, features]
X_train_scaled = scaler.fit_transform(X_train_reshaped).reshape(X_train.shape)

# Escalonamento dos dados de teste
X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])
X_test_scaled = scaler.transform(X_test_reshaped).reshape(X_test.shape)

# Escalonamento das saídas (y)
y_train_reshaped = y_train.reshape(-1, y_train.shape[-1])
y_train_scaled = scaler.fit_transform(y_train_reshaped).reshape(y_train.shape)

y_test_reshaped = y_test.reshape(-1, y_test.shape[-1])
y_test_scaled = scaler.transform(y_test_reshaped).reshape(y_test.shape)

print("Escalonamento dos dados concluído.")
